In [583]:
import pandas as pd
import numpy as np

In [584]:
df = pd.read_csv('data/grade.csv')
df.drop(columns=['student_id'], inplace=True)
X = df.iloc[:, :-2].values
y1 = df.iloc[:, -2].values
y2 = df.iloc[:, -1].values


In [585]:
from sklearn.model_selection import train_test_split
indices = np.arange(len(df))

X_train, X_test, y1_train, y1_test, idx_train, idx_test = train_test_split(
    X, y1, indices, test_size=0.3, random_state=42
)



In [586]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [587]:
X_train

array([[ 1.18661763, -0.02526174,  0.46726365],
       [-0.75408696,  0.38483627, -0.91029037],
       [-1.9416823 , -1.16091776,  0.87542781],
       ...,
       [-0.72512122,  0.70029628, -1.06335192],
       [ 0.1872996 , -0.14093041, -0.34906465],
       [-1.75340499,  0.2691676 ,  2.04889975]], shape=(10458, 3))

In [588]:
from sklearn.preprocessing import MinMaxScaler
sc_y = MinMaxScaler()
y1_train_scaled = sc_y.fit_transform(y1_train.reshape(-1, 1))
y1_test_scaled = sc_y.transform(y1_test.reshape(-1, 1))


In [589]:
y1_train_scaled

array([[1.        ],
       [0.72248244],
       [0.6147541 ],
       ...,
       [0.764637  ],
       [0.89227166],
       [0.51053864]], shape=(10458, 1))

In [590]:
import tensorflow as tf
ann = tf.keras.models.Sequential()

In [591]:
ann.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [592]:
ann.add(tf.keras.layers.Dense(units=64, activation='relu'))

In [593]:
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))

In [594]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

In [595]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

In [596]:
ann.add(tf.keras.layers.Dense(units=1, activation='linear'))

In [597]:
ann.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [598]:
ann.fit(X_train, y1_train_scaled, batch_size=32, epochs=100)

Epoch 1/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 1s 361us/step - loss: 0.0379 - mae: 0.1295
Epoch 2/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 0.0100 - mae: 0.0769
Epoch 3/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 0.0097 - mae: 0.0757
Epoch 4/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step - loss: 0.0098 - mae: 0.0762
Epoch 5/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 0.0098 - mae: 0.0762
Epoch 6/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 345us/step - loss: 0.0097 - mae: 0.0750
Epoch 7/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - loss: 0.0096 - mae: 0.0753
Epoch 8/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - loss: 0.0095 - mae: 0.0742
Epoch 9/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - loss: 0.0096 - mae: 0.0749
Epoch 10/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step - loss: 0.0095 - mae: 0.0745
Epoch 11/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 326us/step - loss: 0.0095 - mae: 0.0743
Epoch 12/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - loss: 0.0095 - m

In [599]:
y1_pred_scaled = ann.predict(X_test)
y1_pred = sc_y.inverse_transform(y1_pred_scaled)
print(np.concatenate((y1_pred.reshape(len(y1_pred),1), y1_test.reshape(len(y1_test),1)),1))

141/141 ━━━━━━━━━━━━━━━━━━━━ 0s 304us/step
[[ 65.81996918  69.1       ]
 [ 79.01130676  75.5       ]
 [ 68.18424225  88.2       ]
 ...
 [ 75.5375824   59.4       ]
 [ 91.47947693  88.8       ]
 [ 98.91239166 100.        ]]


In [600]:
from sklearn.metrics import r2_score
r2 = r2_score(y1_test, y1_pred)
print(f'R² Score: {r2:.4f}')

R² Score: 0.7184


In [601]:
conditions = [
    (y1_pred > 85),
    (y1_pred <= 85) & (y1_pred > 70),
    (y1_pred <= 70) & (y1_pred > 55),
    (y1_pred <= 55) & (y1_pred > 40),
    (y1_pred <= 40)
]

grades = ["A", "B", "C", "D", "F"]

y2_pred = np.select(conditions, np.array(grades, dtype=object), default="F").astype(object)
y2_test = y2[idx_test]
print(np.concatenate((y2_pred.reshape(len(y2_pred),1), y2_test.reshape(len(y2_test),1)),1))

[['C' 'C']
 ['B' 'B']
 ['C' 'A']
 ...
 ['B' 'C']
 ['A' 'A']
 ['A' 'A']]


In [602]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y2_test, y2_pred)
print(cm)
accuracy_score(y2_test, y2_pred)

[[2126  306   11    0    0]
 [ 387  660  123    3    0]
 [  32  293  253   44    0]
 [   0   33  124   61    0]
 [   0    0    3   23    0]]


0.6916555109326193